In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import shutil
import numpy as np
from pathlib import Path
import random
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import math

from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, SpatialDropout2D
from keras.applications import Xception
from keras.optimizers import Adam
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, BatchNormalization, Conv2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
import os
import random
import shutil
from sklearn.model_selection import train_test_split

def split_dataset(input_directory, output_directory, test_size=0.3):
    # Define input directories
    genuine_images_dir = os.path.join(input_directory, 'genuine')
    fake_images_dir = os.path.join(input_directory, 'fake')

    # Function to get all image paths recursively
    def get_image_paths(directory):
        image_paths = []
        for root, _, files in os.walk(directory):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                    image_paths.append(os.path.join(root, file))
        return image_paths

    # Get the paths for genuine and fake images
    genuine_image_paths = get_image_paths(genuine_images_dir)
    fake_image_paths = get_image_paths(fake_images_dir)

    # Combine the genuine and fake image paths
    image_paths = genuine_image_paths + fake_image_paths
    random.shuffle(image_paths)  # Shuffle the image paths

    # Split the dataset into train and validation sets
    train_paths, val_paths = train_test_split(image_paths, test_size=test_size, random_state=42)

    # Create the train and validation directories
    train_genuine_dir = os.path.join(output_directory, 'train', 'genuine')
    train_fake_dir = os.path.join(output_directory, 'train', 'fake')
    val_genuine_dir = os.path.join(output_directory, 'val', 'genuine')
    val_fake_dir = os.path.join(output_directory, 'val', 'fake')
    os.makedirs(train_genuine_dir, exist_ok=True)
    os.makedirs(train_fake_dir, exist_ok=True)
    os.makedirs(val_genuine_dir, exist_ok=True)
    os.makedirs(val_fake_dir, exist_ok=True)

    # Function to copy images to the appropriate directory
    def copy_images(paths, genuine_dir, fake_dir):
        for path in paths:
            image_name = os.path.basename(path)
            if path in genuine_image_paths:
                dest_dir = genuine_dir
            else:
                dest_dir = fake_dir
            shutil.copy(path, os.path.join(dest_dir, image_name))

    # Save the preprocessed images to the train and validation directories
    copy_images(train_paths, train_genuine_dir, train_fake_dir)
    copy_images(val_paths, val_genuine_dir, val_fake_dir)

    print(f"Dataset split completed. Train set: {len(train_paths)}, Validation set: {len(val_paths)}")
    return train_paths, val_paths

# Example usage
input_directory = "/path/to/dataset/"
output_directory = "/path/to/dataset/"
test_size = 0.15789473684  # 30% for validation, 70% for training
train_paths, val_paths = split_dataset(input_directory, output_directory, test_size)

Dataset split completed. Train set: 1600, Validation set: 300


In [ ]:
train_batch_size = 21
val_batch_size = 9

In [ ]:
preprocessed_dataset_path = "/path/to/dataset/"

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)

# Path of the generated train folder, and val folder
train_data_dir = os.path.join(preprocessed_dataset_path, 'train')
val_data_dir = os.path.join(preprocessed_dataset_path, 'val')

# Load data from preprocessed directory for training
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(299,299),
    batch_size=train_batch_size,
    class_mode='binary',
    color_mode='rgb',
    shuffle=True
)

validation_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(299,299),
    batch_size=val_batch_size,
    class_mode='binary',
    color_mode='rgb',
    shuffle=False
)

# Steps per epoch calculation
train_samples = train_generator.samples
val_samples = validation_generator.samples
steps_per_epoch = math.ceil(train_samples / train_batch_size)
validation_steps = math.ceil(val_samples / val_batch_size)

# Create an infinite generator
def recurrent_generator(generator):
    while True:
        for batch in generator:
            yield batch

# Wrap the generators
train_generator_recurrent = recurrent_generator(train_generator)
validation_generator_recurrent = recurrent_generator(validation_generator)

Base Model

In [ ]:
# Load the Xception model
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

# Creation of the Model
model = Model(inputs=base_model.input, outputs=output)

# Compiling of the Model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

Enhanced Model

In [ ]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

# Creation of the model
model = Model(inputs=base_model.input, outputs=output)

# Compiling of the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Define callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
)

In [ ]:
history = model.fit(
    train_generator_infinite,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator_infinite,
    validation_steps=validation_steps,
    epochs=10,
)

In [ ]:
model.save('Main-Model-3(new3).h5')

In [ ]:
%%capture
train_loss, train_accuracy = model.evaluate(train_generator)       # Evaluate the model's train loss and accuracy

In [ ]:
print('Train loss: ', train_loss * 100, '%')
print('Train accuracy: ', train_accuracy * 100, '%')

Train loss:  4.344992712140083 %
Train accuracy:  100.0 %


In [ ]:
%%capture
validation_loss, validation_accuracy = model.evaluate(validation_generator) # Evaluate the model's validation loss and accuracy

In [ ]:
print('Validation loss: ', validation_loss * 100, '%')
print('Validation accuracy: ', validation_accuracy * 100, '%')

Validation loss:  4.90671694278717 %
Validation accuracy:  100.0 %


In [ ]:
# Plotting training_accuracy and validation_accuracy
plt.figure(figsize=(8, 6))
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
plt.plot(history.history['accuracy'], label='training set')
plt.plot(history.history['val_accuracy'], label='validation set')
plt.legend()
plt.savefig('/content/drive/MyDrive/CS Study 2/Visualizations/ROI_4/Accuracy.png')
plt.show()

In [ ]:
# Plotting training_loss and validation_loss
plt.figure(figsize=(8, 6))
plt.title('Training and Validation Loss')
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy(loss)')
plt.plot(history.history['loss'], label='training set')
plt.plot(history.history['val_loss'], label='validation set')
plt.legend()
plt.savefig('/content/drive/MyDrive/CS Study 2/Visualizations/ROI_4/Loss.png')
plt.show()